In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('../../db/data/raw/WUP2018-F12-Cities_Over_300K.xlsx', skiprows=16)
df.head()

,Index,Country Code,Country or area,City Code,Urban Agglomeration,Note,Latitude,Longitude,1950,1955,...,1990,1995,2000,2005,2010,2015,2020,2025,2030,2035
0,1,4,Afghanistan,20001,Herat,NaN,34.348170,62.19967,82.468,85.751,...,183.465,207.190,233.991,275.678,358.691,466.703,605.575,752.910,897.041,1057.573
1,2,4,Afghanistan,20002,Kabul,NaN,34.528887,69.17246,170.784,220.749,...,1549.320,1928.694,2401.109,2905.178,3289.005,3723.543,4221.532,4877.024,5737.138,6760.500
2,3,4,Afghanistan,20003,Kandahar,NaN,31.613320,65.71013,82.199,89.785,...,233.243,263.395,297.456,336.746,383.498,436.741,498.002,577.128,679.278,800.461
3,4,4,Afghanistan,20004,Mazar-e Sharif,NaN,36.709040,67.11087,30.000,37.139,...,135.153,152.629,172.372,206.403,283.532,389.483,532.689,681.531,816.040,962.262
4,5,8,Albania,20005,Tiranë (Tirana),NaN,41.327500,19.81889,84.513,106.932,...,247.270,287.950,335.336,371.803,408.697,449.298,493.712,535.702,565.301,581.626


In [3]:
df_fixed = pd.melt(df, id_vars=['Country Code','Country or area', 'City Code', 'Urban Agglomeration', 'Latitude', 'Longitude'], value_vars=[1950, 1955, 1960, 1965, 1970, 1975, 1980, 1985, 1990, 1995, 2000, 2005, 2010, 2015, 2020])
df_fixed = df_fixed.sort_values(by=['Country Code', 'Urban Agglomeration']).reset_index().drop(columns=['index']).rename(columns={"Country or area": "Country", "Urban Agglomeration": "City", "variable": "Year", "value": "Population_in_Thousands"})
df_fixed["Interpolated"] = False
df_fixed.columns = df_fixed.columns.str.replace(' ', '_')
df_fixed.head()

,Country_Code,Country,City_Code,City,Latitude,Longitude,Year,Population_in_Thousands,Interpolated
0,4,Afghanistan,20001,Herat,34.34817,62.19967,1950,82.468,False
1,4,Afghanistan,20001,Herat,34.34817,62.19967,1955,85.751,False
2,4,Afghanistan,20001,Herat,34.34817,62.19967,1960,89.166,False
3,4,Afghanistan,20001,Herat,34.34817,62.19967,1965,92.715,False
4,4,Afghanistan,20001,Herat,34.34817,62.19967,1970,101.576,False


In [4]:
df_fixed.to_csv("../../db/data/un_city_population_full.csv", index=False)

In [5]:
gdci_country_scope_df = pd.read_csv('../../db/data/GDCI_city_country.csv')
joined_df = df_fixed.merge(right=gdci_country_scope_df, how="inner", on=["City", "Country"])

In [6]:
joined_df.to_csv("../../db/data/un_city_population_in_scope.csv", index=False)